In [2]:
from itertools import combinations
from typing import Callable
from typing import Type
from typing import Tuple
#from collections.abc import Iterable 
#@abstractmethod
#    @property
#    @abstractmethod
# no need to call super
from icecream import ic
    
    
from abc import ABC, abstractmethod
import numpy as np
        
class function(ABC):
    @property
    @abstractmethod
    def forward_func(self) -> Callable:
        pass
    
    @property
    @abstractmethod
    def backward_func(self) -> Callable:
        pass
    
    def get_functions(self) -> Tuple[Callable, Callable]:
        return self.forward_func, self.backward_func
    
    def __call__(self, *inner) -> expr_node: # simplifies the syntax for chaning of functions
        return expr_node(self, inner)
        
    

class sin(function):
    forward_func = np.sin
    backward_func = np.cos

class tanh(function):
    forward_func = np.tanh
    backward_func = lambda x: 1 - np.square(tanh(x))

class cos(function):
    forward_func = np.cos
    backward_func = lambda x: -np.sin(x)
          

class multiply(function):
    
    forward_func = lambda: None # to be overwitten in constructor
    backward_func = lambda: None # to be overwitten in constructor
    
    def __init__(self,*, allow_arbitrary_many = False): # * makes allow_arbitary_many to keword only
        if not allow_arbitrary_many: # simple case two factors
            self.forward_func = np.multiply
            self.backward_func = lambda x, y: (y, x)
        else:
            forward_func = lambda *x: np.prod(np.vstack(x), axis=0)
            self.backward_func = lambda *x: (np.prod(np.vstack(values), axis=0) for values in combinations(x, len(x) - 1))
                   
            
        
class add(function):
    
    forward_func = lambda: None # to be overwitten in constructor
    backward_func = lambda: None # to be overwitten in constructor
    
    def __init__(self,*, allow_arbitrary_many = False): # new is clalled before the constructer (before: "__init__")
        if not allow_arbitrary_many:
            self.forward_func = np.add 
            self.backward_func = lambda x, y: (np.full_like(x, 1), np.full_like(y, 1)) #arrays with same shape of x and y, filled with 1
        else:
            self.forward_func = lambda *x: np.sum(np.vstack(x), axis=0)
            self.backward_func = lambda *x: (np.full_like(x[0], 1) for _ in range(len(x))) #backward_func = lambda *x: (np.ones(len(x[0]) if isinstance(x[0], Iterable) else 1) for _ in range(len(x))))

            
                    
            
class matrix_vector_product(function):
    def matrix_vector_backwards(self, matrix, vector):
        c = np.zeros((len(matrix), len(matrix), len(vector)))
        for i in range(len(matrix)):
            c[i,i] = vector
        return (c, matrix) 
    forward_func = np.matmul
    backward_func = matrix_vector_backwards
    
class vector_vector_sum(function):
    #vectors must have same length
    forward_func = np.add
    backward_func = lambda v1, v2: (np.eye(len(v1)), np.eye(len(v2)))
    
    
    
#b = function2(np.sin, np.cos)
c = sin()
print(c, c.forward_func, c.backward_func)
f1, f2 = c.get_functions()
print(f1, f2)
#c.eval_forward(2)
d = [sin(), sin()]


e = multiply()
print(e.forward_func)


e = multiply(allow_arbitrary_many=True)
print(e.forward_func)
e = multiply(allow_arbitrary_many=False)
print(e.forward_func)
f = cos()
print(f.forward_func(2), np.cos(2))  
forw, backw = c.get_functions()
print(forw, backw) 
ic.disable()
ghj = ic(f.forward_func(2))
ic(ghj)
k = tanh()
l = cos()
ic.enable()
ic(l.forward_func(2))
ic()




        

NameError: name 'expr_node' is not defined

In [250]:
from collections.abc import Sequence
from __future__ import annotations
from numbers import Number

class expr_end_node():
    def __init__(self, value: Number | np.ndarray , grad_value: Number | np.ndarray  = 0):
        self.value = value
        self.grad_value = grad_value
        
class expr_node():
    def __init__(self, func: function, childs: Sequence[expr_node | expr_end_node] = []):
        #self.parants = parants #expr_node
        self.childs = childs  #expr_node
        self.func = func #function 
        # lieber so:
        # self.forward_func, self.backward_func = func.get_functions()
        # und dann self.func entfernen
        
f = sin()
g = sin()
ex1 = expr_node(f)
ex2 = expr_node(g)
ex3 = expr_node(g)
ex1.childs = [ex2]
#ex2.parants = [ex1]
ex2.childs = [ex3]
#sin(sin(sin()))








In [251]:
ex1 = expr_node(sin())
ex2 = expr_node(sin())
ex3 = expr_node(sin())
x = expr_end_node(np.float64(0.345))
ex1.childs = [ex2]
ex2.childs = [ex3]
ex3.childs = [x]

#sin(sin(sin(x)))
"""
sin
 |
sin
 |
sin
 |
 x=0.345
"""


#cos(a)*sin(b + tanh(c))
example_expr2 = expr_node(multiply(), 
                          [expr_node(cos(),
                                     [expr_end_node(np.float64(0.345))]
                                    ), 
                           expr_node(sin(),
                                     [expr_node(add(), 
                                                [expr_end_node(np.float64(0.345)),
                                                 expr_node(tanh(), 
                                                           [expr_end_node(np.float64(0.345))]
                                                          )
                                                ]
                                               )
                                     ]
                                    )
                          ]
                         )

# the same using the __call__ method of function (see above in function definition):
example_expr3 = multiply()(
                        cos()(
                            expr_end_node(np.float64(0.345))
                        ),
                        sin()(
                            add()(
                                expr_end_node(np.float64(0.345)),
                                tanh()(
                                    expr_end_node(np.float64(0.345))
                                )
                            )
                        )
                )
"""
    multiply
    |      |
   cos    sin
   |       |
  a=0.345 add
          | |
    b=0.345 tanh
             |
         c=0.345
"""
#short version of the sin(sin(sin(x))) exaple:
x = expr_end_node(np.float64(0.345))
example_expr1 = sin()( sin()( sin()( x ) ) ) 

x1 = expr_end_node(np.float64(0.345))
x2 = expr_end_node(np.float64(0.345))
x3 = expr_end_node(np.float64(0.345))
x4 = expr_end_node(np.float64(0.345))
sin()(multiply())
#sin(x1*x2*cos(x3*x4))





In [252]:
#!source .jupyter_venv/.venv/bin/activate # source venv, if you have one for your kernal
#import sys
#!{sys.executable} -m pip install graphviz

import graphviz


def print_graph(node, graph, parent_id=""):
    
    node_id = str(id(node))
    node_label = str(node.value) if isinstance(node, expr_end_node) else type(node.func).__name__
    graph.node(node_id, node_label)  
    
    if parent_id:
        graph.edge(parent_id, node_id)
        
    if hasattr(node, 'childs') and node.childs:
            for child in node.childs:
                print_graph(child, graph, node_id)#, constraint='false')


dot1 = graphviz.Digraph('the_graph', comment='test') 
dot1.attr(rankdir="LR")
print_graph(example_expr3, dot1)
#doctest_mark_exe() 
dot1.render(directory='graph_out/tt', view=True)
#dot  
#str(id(ex1))
#type(ex1.func).__name__

'graph_out/tt/the_graph.gv.pdf'

In [1]:

#f(g(h(x)))' = f'(g(h(x))) * (g(h(x)))' =  f'(g(h(x))) * g'(h(x)) * h'(x)
# h'(x) * g'(h(x)) * f'(g(h(x)))


def forward(node):
    return node.func.forward_func(*(forward(child) for child in node.childs)) if type(node) is not expr_end_node else node.value 
forward(ex1)
"""
def forward(node):
    if type(node) is not expr_end_node:
    
        node.func.forward_func(*(forward(child) for child in node.childs))
    else:
        node.value 
"""
forward(ex1)

def backward(node, value = 1):
    if type(node) is not expr_end_node:
        child_values = [forward(child) for child in node.childs]
        if len(node.childs) == 1:
            backward(node.childs[0], value * node.func.backward_func(*child_values))
        else:
            for child, new_value in zip(node.childs, node.func.backward_func(*child_values), strict=True):
                backward(child, value * new_value)
    else: 
        node.grad_value += value
        
#result,target
    
        
        
x.grad_value=0

ic(forward(ex1))
ic(np.sin(np.sin(np.sin(np.float64(.345)))))

backward(ex1)
ic(x.grad_value)
ic(np.cos(np.sin(np.sin(.345)))*np.cos(np.sin(.345))*np.cos(0.345))

print(x.grad_value)


"""
def print_all(a,b,c):
    print(a)
    print(b)
    print(c)

print_all(*[1,2,3])
"""
#zip((a1,a2.a3),(b1,b2,b3) -> ((a1,b1),(a2,b2),(a3,b3))

x1 = expr_end_node(np.float64(3.11))
x2 = expr_end_node(np.float64(1.11))
b = multiply()(x1,x2)
ic(forward(b))
backward(b)
ic(x1.grad_value, x2.grad_value)

mat = expr_end_node(np.random.rand(3,2))
vec = expr_end_node(np.random.rand(2))
a = matrix_vector_product()(mat, vec)
forward(a)
backward(a)
ic(mat.value, vec.value)
ic(mat.grad_value, vec.grad_value )







NameError: name 'ex1' is not defined

In [290]:
from sklearn.datasets import load_iris
from numpy import linalg as LA



iris = load_iris()
print(len(iris.data), len(iris.target))
print(type(iris.data), type(iris.target))
ic(iris.target)
normalized_data = iris.data/LA.norm(iris.data, axis=0)

#%timeit LA.norm(iris.data, axis=0)
#%timeit LA.norm(iris.data,np.inf, axis=0)
#%timeit np.max(iris.data, axis=0)

ic(LA.norm(iris.data, axis=0))
ic(LA.norm(iris.data,np.inf, axis=0))
ic(np.max(iris.data, axis=0))
"""
out = vector len 3 , out_1: suit 1,  out_1: suit 2, 
suit_1 = (1,0,0)
suit_2 = (0,1,0)
suit_3 = (0,0,1)

netz = 
    in: vecor len 4 
    out = vector len 3
    
with  2-5 hidden layer; 4- 10 neurons each
"""

ic| iris.target: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                        2, 2, 2, 2, 

150 150
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
ic| LA.norm(iris.data, axis=0): array([72.27620632, 37.82062929, 50.82036993, 17.38763929])
ic| LA.norm(iris.data,np.inf, axis=0): array([7.9, 4.4, 6.9, 2.5])
ic| np.max(iris.data, axis=0): array([7.9, 4.4, 6.9, 2.5])


'\nout = vector len 3 , out_1: suit 1,  out_1: suit 2, \nsuit_1 = (1,0,0)\nsuit_2 = (0,1,0)\nsuit_3 = (0,0,1)\n\nnetz = \n    in: vecor len 4 \n    out = vector len 3\n    \nwith  2-5 hidden layer; 4- 10 neurons each\n'

In [10]:
test_prediction = np.array([0.2, 0.6, 0.3])
test_target = np.array([0, 1, 0])

# Mean squared error (MSE):
def loss_mse(prediction, target):
    return np.square(prediction - target).sum()
#(pred_x, pred_y), (target_x, target_y) -> (pred_x - target_x)²+(pred_y - target_y)²



loss1 = ic(loss_mse(test_prediction, test_target))
ic(np.log(test_prediction))
#loss2 = ic(loss_ce(test_prediction,test_target))

#ic.disable()
#ic.enable()

# 0.2²+0.4²+0.3² = 
#ic(loss1(test_prediction, test_target))

ic| loss_mse(test_prediction, test_target): np.float64(0.29000000000000004)
ic| np.log(test_prediction): array([-1.60943791, -0.51082562, -1.2039728 ])


array([-1.60943791, -0.51082562, -1.2039728 ])

In [9]:
in_1     out_1
in_2     out_2

initial = w, b are random values
netz(in) = 
    out_1 = in_1 * w_{1,1} + in_2 *w_{1,2} + b_1
    out_2 = in_1 * w_{2,1} + in_2 *w_{2,2} + b_2

# net(in, {w}, {b}) 
out = netz(in)
loss(out, true_val)
backpropergation -> change  w, b <--------



dict(np.sin: np.cos)

SyntaxError: invalid syntax (1707078846.py, line 1)

In [8]:
def prin_list(a, b, *x):
        print(*x)

In [10]:
prin_list(1,25555555555)

In [13]:
!source .jupyter_venv/.venv/bin/activate # source venv, if you have one for your kernal
import sys

!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

/bin/bash: Zeile 1: .jupyter_venv/.venv/bin/activate: Datei oder Verzeichnis nicht gefunden
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.2 MB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Using cached https://download.pytorch.org/whl/setuptools-70.0.0-py3-none-any.whl (863 kB)
  Using cached https://download.pytorch.org/whl/sym

In [15]:
import torch
import math

dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 815.4979248046875
199 579.5009155273438
299 412.5529479980469
399 294.4494934082031
499 210.89791870117188
599 151.7886199951172
699 109.97039794921875
799 80.38469696044922
899 59.45298767089844
999 44.64371871948242
1099 34.16594314575195
1199 26.752681732177734
1299 21.50754165649414
1399 17.796405792236328
1499 15.170598983764648
1599 13.312688827514648
1699 11.998108863830566
1799 11.067950248718262
1899 10.409802436828613
1999 9.944101333618164
Result: y = 0.03551137074828148 + 0.8578885197639465 x + -0.006126299500465393 x^2 + -0.09349364042282104 x^3


In [222]:
import torch
import math

dtype = torch.float

#input
x = torch.tensor([0, 1], dtype = dtype)
#target
y_target = torch.tensor([1, 0], dtype = dtype)

#parameters
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)

# forward_function
def forward_function(x, a, b):
    # y1 = a+b*x1
    # y2 = a+b*x2
    # ....
     return a + b * x
    
# (d forward_function)/(d a) = (1, 1, ...)
def a_backwars(x, a, b):
     return torch.ones(x.shape, dtype = dtype)
     #return torch.tensor([1 for _ in range(len(forward_function(x, a, b)))], dtype = dtype)
    
# (d forward_function)/(d b) = (x1, x2, ...)
def b_backwars(x, a, b):
     return x
    
# loss_function   
def loss_function(y_pred, y_target):
     return (y_pred - y_target).pow(2).sum()
    
# (d loss)/(d y_pred) = 2*(y_pred - y_target)
def loss_backwards(y_pred, y_target):
     return 2*(y_pred - y_target)
    
ic(forward_function(x, a, b))
y_pred = forward_function(x, a, b)
loss = loss_function(y_pred, y_target)

# Use autograd to compute the backward pass. This call will compute the
# gradient of loss with respect to all Tensors with requires_grad=True.
# After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
# the gradient of the loss with respect to a, b, c, d respectively.
loss.backward()




ic(" ")
ic("backprop. of a (manually):", (loss_backwards(y_pred, y_target)@a_backwars(x, a, b)).item())
ic("backprop. of a (with autograd):", a.grad)
ic(" ")
ic("backprop. of a (manually):", (torch.t(a_backwars(x, a, b))@loss_backwards(y_pred, y_target)).item())
ic("backprop. of a (with autograd):", a.grad)
ic(" ")
ic("backprop. of b (manually):", (loss_backwards(y_pred, y_target)@b_backwars(x, a, b)).item())
ic("backprop. of b (with autograd):", b.grad)
ic(" ")
ic("backprop. of b (manually):", (torch.t(b_backwars(x, a, b))@loss_backwards(y_pred, y_target)).item())
ic("backprop. of b (with autograd):", b.grad)
ic(b.grad)
ic(" ")




ic| forward_function(x, a, b): tensor([-0.7378, -1.2779], grad_fn=<AddBackward0>)
ic| ' '
ic| "backprop. of a (manually):": 'backprop. of a (manually):'
    (loss_backwards(y_pred, y_target)@a_backwars(x, a, b)).item(): -6.031282424926758
ic| 'backprop. of a (with autograd):', a.grad: tensor(-6.0313)
ic| ' '
ic| "backprop. of a (manually):": 'backprop. of a (manually):'
    (torch.t(a_backwars(x, a, b))@loss_backwards(y_pred, y_target)).item(): -6.031282424926758
ic| 'backprop. of a (with autograd):', a.grad: tensor(-6.0313)
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    (loss_backwards(y_pred, y_target)@b_backwars(x, a, b)).item(): -2.5557620525360107
ic| 'backprop. of b (with autograd):', b.grad: tensor(-2.5558)
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    (torch.t(b_backwars(x, a, b))@loss_backwards(y_pred, y_target)).item(): -2.5557620525360107
ic| 'backprop. of b (with autograd):', b.grad: tensor(-2.5558)
ic| b.grad: tensor

' '

In [232]:

dtype = torch.float

#input
#x = torch.tensor([0, 1], dtype = dtype)
x = torch.randn((2), dtype=dtype, requires_grad=True)

#target
#y_target = torch.tensor([1, 0], dtype = dtype)
y = torch.randn((2), dtype=dtype, requires_grad=True)

#parameters
#w = torch.arange(4).reshape(2,2)
#w = torch.tensor(w.clone(), dtype=dtype, requires_grad=True)
#b = torch.arange(2, dtype=dtype, requires_grad=True)

w = torch.randn((2,2), dtype=dtype, requires_grad=True)
b = torch.randn((2), dtype=dtype, requires_grad=True)

# forward_function
def forward_function(x, w, b):
    # y1 = b1+w(1,1)*x1 + w(1,2)*x2
    # y2 = b2+w(2,1)*x1 + w(2,2)*x2
     return w @ x + b
    
# (d forward_function)/(d w) = (1, 1, ...)
#def w_backwars(x, w, b):
#     return torch.ones(len(x), dtype = dtype)
    
# (d forward_function)/(d b) = (x1, x2, ...)
def b_backwars(x, w, b):
    #(d y)/(d b) = ((d y1)/(d b), (d y2)/(d b)) 
    # = (((d y1)/(d b1), (d y1)/(d b2)), ((d y2)/(d b1), (d y2)/(d b2))) #Jacobimatrix
    # = ((1, 0), (0, 1))
    return torch.eye(len(x), dtype = dtype)

def w_backwars(x, w, b):
    # (d y)/(d w) = ((d y1)/(d w), (d y2)/(d w)) 
    # = ((((d y1)/(d w(1,1)), (d y1)/(d w(1,2))), ((d y1)/(d w(2,1)), (d y1)/(d w(2,2)))), (((d y2)/(d w(1,1)), (d y2)/(d w(1,2))), ((d y2)/(d w(2,1)), (d y2)/(d w(2,2))))) #Von uns veralgemeinerte Jacobimatrix (Tensor 3.stufe)
    # = ((( x1              ,  x2              ), ( 0               ,  0               )), (( 0               ,  0               ), ( x1              ,  x2              )))
    # = ([(x1, x2), (0,  0)], [(0,  0), (x1,  x2)])
    # 
    c = torch.zeros(len(x), len(x), len(x), dtype=x.dtype) 
    for i in range(len(x)):
        c[i,i] = x
    return c
    #return x.repeat(len(x),1) # does not work
    
    
# loss_function   
def loss_function(y_pred, y_target):
     return (y_pred - y_target).pow(2).sum()
    
# (d loss)/(d y_pred) = 2*(y_pred - y_target)
def loss_backwards(y_pred, y_target):
     return 2*(y_pred - y_target)
    
y_pred = forward_function(x, w, b)
loss = loss_function(y_pred, y_target)

# Use autograd to compute the backward pass. This call will compute the
# gradient of loss with respect to all Tensors with requires_grad=True.
# After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
# the gradient of the loss with respect to a, b, c, d respectively.
loss.backward()

ic(loss_backwards(y_pred, y_target),w_backwars(x, w, b) )
test = w_backwars(x, w, b)[0] * loss_backwards(y_pred, y_target)[0] + w_backwars(x, w, b)[1] * loss_backwards(y_pred, y_target)[1]
ic(test)
ic(" ")
ic("backprop. of w (manually):", (loss_backwards(y_pred, y_target)@w_backwars(x, w, b)))
ic("backprop. of w (with autograd):", w.grad)
ic(" ")
ic("backprop. of b (manually):", (loss_backwards(y_pred, y_target)@b_backwars(x, w, b)))
ic("backprop. of b (with autograd):", b.grad)
#ic(b.grad)
ic(" ")

ic| loss_backwards(y_pred, y_target): tensor([-0.6472,  1.5129], grad_fn=<MulBackward0>)
    w_backwars(x, w, b): tensor([[[-0.3763,  0.3583],
                                  [ 0.0000,  0.0000]],
                         
                                 [[ 0.0000,  0.0000],
                                  [-0.3763,  0.3583]]], grad_fn=<CopySlices>)
ic| test: tensor([[ 0.2435, -0.2319],
                  [-0.5693,  0.5420]], grad_fn=<AddBackward0>)
ic| ' '
ic| "backprop. of w (manually):": 'backprop. of w (manually):'
    loss_backwards(y_pred, y_target)@w_backwars(x, w, b): tensor([[ 0.2435, -0.2319],
                                                                  [-0.5693,  0.5420]], grad_fn=<UnsafeViewBackward0>)
ic| "backprop. of w (with autograd):": 'backprop. of w (with autograd):'
    w.grad: tensor([[ 0.2435, -0.2319],
                    [-0.5693,  0.5420]])
ic| ' '
ic| "backprop. of b (manually):": 'backprop. of b (manually):'
    loss_backwards(y_pred, y_target)@b_back

' '

In [220]:
a = torch.randn((2,2), dtype=dtype, requires_grad=True)
b = torch.randn((2), dtype=dtype, requires_grad=True)
ic(a)
ic(b)

ic| a: tensor([[-1.9353, -1.2393],
               [ 2.0114, -0.1456]], requires_grad=True)
ic| b: tensor([ 0.8283, -0.0090], requires_grad=True)


tensor([ 0.8283, -0.0090], requires_grad=True)

In [210]:
a = torch.randn(2, 3)

b = torch.eye(2)
ic(b)
a = torch.tensor([1,2,3])
c = torch.tensor([[[1, 2],[0, 0]], [[0, 0], [1, 2]]])
#c = torch.stack([a.expand(2, 2), torch.diag(a)])
c = torch.zeros(len(a), len(a), len(a), dtype=a.dtype) 
for i in range(len(a)):
    c[i,i] = a

c = a.repeat(len(a),1)

print(c)
#ic(torch.outer(b, a))
print(a)
print(torch.diag_embed(a))
print(torch.diag(a))


ic| b: tensor([[1., 0.],
               [0., 1.]])


tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])
tensor([1, 2, 3])
tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])
tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])


In [30]:
#!source .jupyter_venv/.venv/bin/activate # source venv, if you have one for your kernal
import sys

!{sys.executable} -m pip install pyhafas
!{sys.executable} -m pip install pandas

In [287]:
list = [i for i in range(3)]
print(*list)

0 1 2
